# Chapter 15 - Classifying Images with Deep Convolutional Neural Networks

## Optimization
... is a research subject of its own. Some simple techniques are much used in Deep Learning:  
- Momentum
- Nesterov Momentum
- Decay  
  
We will use code examples with simple linear regression adapted from https://github.com/jsphbtst/linear-regression-optimization-technique.

In [16]:
# Linear regression using gradient descent
# Parameters m: slope and b: intercept, cost function MSE
def LinearRegression(X, y, epochs=1000, learning_rate=0.0001, activation="gradient_descent", batch_size=256, 
                     mu=0.9, m_current=0, b_current=0):
    N = float(len(y))
    mini_batch_cost = []

    if activation == "gradient_descent":
        for i in range(epochs):
            # Prediction
            y_current = (m_current * X) + b_current            
            
            # Cost (MSE)
            cost = sum([data**2 for data in (y-y_current)]) / N

            # Derivative of cost function
            m_gradient = -(2/N) * sum(X * (y - y_current))
            b_gradient = -(2/N) * sum(y - y_current)

            # Weight update
            m_current = m_current - (learning_rate * m_gradient)
            b_current = b_current - (learning_rate * b_gradient)

In [ ]:
# Stochastic / mini-batch alternative
    elif activation == "sgd":
        for i in range(epochs):
            for j in range(0, int(N), batch_size):
                # Prediction of current batch:
                y_current = (m_current * X[j:j+batch_size]) + b_current

                # Cost of current batch:
                mini_batch_cost.append(sum([data**2 for data in (y[j:j+batch_size] - y_current)]) / N)

                # Gradient of current batch
                m_gradient = -(2/N) * sum(X[j:j+batch_size] * (y[j:j+batch_size] - y_current))
                b_gradient = -(2/N) * sum(y[j:j+batch_size] - y_current)

                # Weight updates
                m_current = m_current - (learning_rate * m_gradient)
                b_current = b_current - (learning_rate * b_gradient)

            # Cost of epoch:
            cost = sum(mini_batch_cost) / float(len(mini_batch_cost))
            mini_batch_cost = []

### Momentum
- The _classical momentum_ method (Polyak, 1964) is a technique for accelerating gradient descent that accumulates a velocity vector in directions of persistent reduction in the objective across iterations.
    - "Successful updates of weights are given an extra push."
    - Momentum affects convergence most in the "transient phase", i.e. before fine tuning.
- Think of a ball rolling down a hill
    - When slope is negative in the direction of rolling, momentum increases
    - When slope changes twists or is positive, momentum decreases
$$\nu_t = \gamma \nu_{t-1} + \eta \triangledown_\theta J(\theta),$$
where $\gamma$ is often around 0.9.
- $\nu_{t-1}$ is the memory from previous iterations, and $\gamma$ controls how much is remembered

In [ ]:
# SGD / mini-batch with Momentum alternative
    elif activation == "momentum":
        for i in range(epochs):
            for j in range(0, int(N), batch_size):
                # Prediction of current batch:
                y_current = m_current * X[j:j+batch_size] + b_current

                # Cost of current batch:
                mini_batch_cost.append(sum([data**2 for data in (y[j:j+batch_size] - y_current)]) / N)

                # Gradient of current batch
                m_gradient = -(2/N) * sum(X[j:j+batch_size] * (y[j:j+batch_size] - y_current))
                b_gradient = -(2/N) * sum(y[j:j+batch_size] - y_current)

                # Reset momentum for each epoch
                if i == 0:
                    v_m = 0
                    v_b = 0

                # Momentum update (previous accumulation + fresh gradient)
                v_m = mu * v_m + learning_rate * m_gradient
                v_b = mu * v_b + learning_rate * b_gradient

                # Weight update
                m_current = m_current - v_m
                b_current = b_current - v_b        

            # Cost of epoch:
            cost = sum(mini_batch_cost) / float(len(mini_batch_cost))
            mini_batch_cost = []

### Nesterov momentum
- Momentum may still be large near the optimum (imagine the ball again)
- _Nesterov momentum_ adds a partial update of the gradient before adding momentum to the weight update.
    - "Successful updates of weights are given two extra pushes."
    - First push is anticipatory, guessing a likely gradient update
$$\nu_t = \gamma \nu_{t-1} + \eta \triangledown_\theta J(\theta - \gamma \nu_{t-1})$$
<img src="./images/Nesterov.jpeg" alt="Momentum" style="width: 500px;"/>
Source: http://cs231n.github.io/assets/nn3/nesterov.jpeg

- Explanation and Python examples: https://towardsdatascience.com/a-bit-beyond-gradient-descent-mini-batch-momentum-and-some-dude-named-yuri-nesterov-a3640f9e496b

In [ ]:
# SGD / mini-batch with Nesterov Momentum alternative
    elif activation == "nesterov":
        for i in range(epochs):
            for j in range(0, int(N), batch_size):
                # Prediction of current batch:
                y_current = (m_current * X[j:j+batch_size]) + b_current

                # Cost of current batch:
                mini_batch_cost.append(sum([data**2 for data in (y[j:j+batch_size] - y_current)]) / N)

                # Reset momentum for each epoch
                if i == 0:
                    v_m = 0
                    v_b = 0

                # Nesterov step
                y_nesterov_m = (m_current - mu * v_m) * X[j:j+batch_size] + b_current
                y_nesterov_b = (b_current - mu * v_b) * X[j:j+batch_size] + b_current

                # Gradient of current batch with Nesterov step
                m_gradient = -(2/N) * sum(X[j:j+batch_size] * (y[j:j+batch_size] - y_nesterov_m))
                b_gradient = -(2/N) * sum(y[j:j+batch_size] - y_nesterov_b)

                # Momentum update
                v_m = mu * v_m + learning_rate * m_gradient
                v_b = mu * v_b + learning_rate * b_gradient

                # Weight update
                m_current = m_current - v_m
                b_current = b_current - v_b

            cost = sum(mini_batch_cost) / float(len(mini_batch_cost))
            mini_batch_cost = []

    else:
        raise Exception("ERROR: Activation Function Not Found!")

    return m_current, b_current, cost            

### Decay
- Where momentum accellerates in promising directions, decay reduces the learning rate.
- Finetuning of models requires small steps.
    - Shrink the learning rate gradually
    - For instance $lr_{epoch ~ i+1} = lr_{epoch ~ i} \times 0.99$
- Too fast decay => never reach minimum
- Decay with restart and more adaptive decays may be useful
- [Reduce learning rate on plateaus](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau)

### Optimizers
- AdaGrad: Parameter-specific learning rates, which are adapted relative to how frequently a parameter gets updated during training. 
    -  It adapts the learning rate to the parameters, performing smaller updates
    - AdaDelta: Same idea, but in a local window of iterations (not accumulated over all updates).
- RMSProp: Divide the gradient/learning rate for each weight by a running average of their recent, individual magnitudes - [Lecture by G.F.Hinton](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)

### Optimizers ctd.
- Adam is a robust gradient-based optimization method inspired by RMSProp and AdaGrad,
    - suited for nonconvex optimization and machine learning problems,
    - choice of update step size derived from the running average of gradient moments.
    - Used to have guaranteed convergence, but now reduced to practically always converges.
- SGD with momentum also popular (no invidual learning rates)

<img src="./images/Optimization1.gif" alt="Optimization" style="width: 600px;"/>  
Image credit [Alec Radford](https://twitter.com/alecrad)

<img src="./images/Optimization2.gif" alt="Optimization" style="width: 600px;"/>  
Image credit [Alec Radford](https://twitter.com/alecrad)  